

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_RESTAURANT.ipynb)






# **Ner_Restaurant**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


## 2. Some sample examples

In [4]:
text_list_restaurant = ["""Noma is located on the north side of the city. The restaurant has a cuisine based solely on ingredients found in the area. These days, \
dinner may begin with potato pie and fresh fruit juice. The meal can then continue with traditional Danish pastry. \
This wonderful borek is brushed with a garlic sauce and finished with almond, hazelnut and plum puree.It sounds wacky, but somehow crew \
manage to make it all delicious.It also has the feature of being a vegan restaurant. I think it deserves 5 stars.The price of a meal for two is \
very reasonable for this taste. The restaurant, which is open until 2 am, is always very crowded.""",


"""This restaurant,  favorite pasta bar also offers one of the most reasonably priced lunch sets in town! With locations spread out all over \
the territory Sha Tin – Pici’s formidable lunch menu reads like a highlight reel of the restaurant. Choose from starters like the burrata and \
arugula salad or freshly tossed tuna tartare, and reliable handmade pasta dishes like pappardelle. Finally, round out your effortless Italian \
meal with a tidy one-pot tiramisu, of course, an espresso to power you through the rest of the day.""",


"""My favorite dish at the Oceandelf Restaurant, was shrimp and grits topped with a tasty tasso sauce and loaded with locally netted Georgia \
white shrimp. The chef and his staff are staunch advocates of the farm-to-table concept, sourcing produce, meats,poultry and dairy products \
almost exclusively from Georgia organic farms.They offer a wide array of fresh food – green pork plate, hamburger, barbacoa plate,  bbq with \
rice and beans and more.They can be one of the best representatives of Georgian cuisine. They prefer the freshest ingredients when preparing \
their meals to ensure the best quality and taste. Its prices are very affordable compared to almost other places, it appeals to everyone.Since \
they are open until 11 pm, they are a great alternative for late evening meals.""",


"""Ahloven Restaurant opened near North Avenue in 2000. I visited for lunch with a friend. We enjoyed the authentic South Indian main courses. \
The menu was good. The brownie after the meal was very good.The service was above average but the fries were too flavorless to finish. For  \
15 dollars the quality should have been much higher. When the waiter came to our table and asked if we wanted anything, we explained that \
we didn't like the french fries. He was understanding and offered us a discount on the fries.""",


"""Mcdills restaurant is situated in the heart of a semi-urban area near a motorsports academy. It is possible to find fresh and tasty food \
every hour in the restaurant, which is open until 3 am. The menu offers a wide variety of delicious. I had smoked salmon with horseradish cream, \
as well as prawns and lime vinaigrette. My friend tried the triple cheese and tarragon-stuffed mushrooms. They were all exquisite.The main course \
included a fantastic veggie shepherd’s pie with sweet potato mash and creamy courgette lasagne. It was all followed by a savory apple pie with \
a chocolate. Everything we ate was fresh and delicious. The award-winning chefs use the best quality ingredients to produce dishes that are simply \
delightful. The restaurant also offers  a wonderful drink menu, and they offer one of the best wines in the region. In contrast, however, the \
service was not of the highest quality. It seemed to me that the waiters were not very engaged or interested in their jobs.He came around to \
check on our experience and seemed to be authentically engaged.Prices are quite expensive compared to other places."""]

## 3. Define Spark NLP pipeline

In [5]:
document = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")  

embeddings = WordEmbeddingsModel.pretrained("glove_100d")\
  .setInputCols("sentence", "token") \
  .setOutputCol("embeddings")

nerdl = NerDLModel.pretrained("nerdl_restaurant_100d")\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setOutputCol("ner")

ner_converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")

pipeline = Pipeline(
    stages=[
        document,
        sentenceDetector,
        tokenizer,
        embeddings, 
        nerdl, 
        ner_converter])

df = spark.createDataFrame(text_list_restaurant, StringType()).toDF("text")
result = pipeline.fit(df).transform(df)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
nerdl_restaurant_100d download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


## 4. Visualize results

In [6]:
from sparknlp_display import NerVisualizer

for i in range(len(text_list_restaurant)):
  NerVisualizer().display(
      result = result.collect()[i],
      label_col = 'ner_chunk',
      document_col = 'document'
  )